<a href="https://colab.research.google.com/github/Karasiari/Graphs/blob/main/Graphs_intial%2Banalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import os # Выберите, откуда тяните папку
from google.colab import files

# По папке формируем словарь графов со структурой -
#                                                {'название графа (как в папке)':
#                                                       {'adj_matrix': матрица смежности,
#                                                        'traffic_matrix': матрица корреспонденций,
#                                                        'nodes_params': {'degrees': список степеней вершин,
#                                                                         'source_demands': список суммарного входящего траффика,
#                                                                         'target_demands': список суммарного исходящего траффика,
#                                                                         'sum_demands': список усредненного траффика (source + target) / 2
#                                                                        }
#                                                       }
#                                                }

def csv_to_graph(path, demands_path):
  Df = pd.read_csv(path, header=None, names = ['id', 'source', 'target', 'length'])
  Traffic = pd.read_csv(demands_path, header=None, names = ['id', 'source', 'target', 'bitrate'])
  df = Df.iloc[1:].copy()
  traffic = Traffic.iloc[1:].copy()
  df['source'] = df['source'].astype(int)
  df['target'] = df['target'].astype(int)
  df['length'] = df['length'].astype(float)
  traffic['source'] = traffic['source'].astype(int)
  traffic['target'] = traffic['target'].astype(int)
  traffic['bitrate'] = traffic['bitrate'].astype(int) // 100
  traffic_grouped = traffic.groupby(['source', 'target']).sum().reset_index()

  sources = df['source'].tolist()
  targets = df['target'].tolist()
  lengths = df['length'].tolist()

  unique_vertices = set()
  for source, target in zip(sources, targets):
    unique_vertices.add(source)
    unique_vertices.add(target)
  vertex_mapping = {old: new for new, old in enumerate(unique_vertices)}
  num_vertices = len(unique_vertices)

  adj_matrix = np.zeros((num_vertices, num_vertices))
  traffic_matrix = np.zeros((num_vertices, num_vertices))

  for source, target, length in zip(sources, targets, lengths):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    adj_matrix[new_source, new_target] = length
    adj_matrix[new_target, new_source] = length

  traffic_grouped['source'] = traffic_grouped['source'].map(vertex_mapping)
  traffic_grouped['target'] = traffic_grouped['target'].map(vertex_mapping)
  for _, row in traffic_grouped.iterrows():
    traffic_matrix[row['source'], row['target']] = row['bitrate']

  nodes_params = {}

  degrees = list(np.count_nonzero(adj_matrix, axis=1).astype(np.float64))
  nodes_params['degrees'] = degrees

  source_demands = list(np.sum(traffic_matrix, axis=1))
  nodes_params['source_demands'] = source_demands

  target_demands = list(np.sum(traffic_matrix, axis=0))
  nodes_params['target_demands'] = target_demands

  sum_demands = [(i + j)/2 for (i,j) in zip(source_demands, target_demands)]
  nodes_params['sum_demands'] = sum_demands

  return (adj_matrix, traffic_matrix, nodes_params)

def get_graphs(base_path):
  Graphs = {}
  csv_tables = []
  for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)
    csv_path = os.path.join(folder_path, 'links.csv')
    csv_path_demands = os.path.join(folder_path, 'demands.csv')
    csv_tables.append(folder_name)

    (adj_matrix, traffic_matrix, nodes_params) = csv_to_graph(csv_path, csv_path_demands)
    Graphs[folder_name] = {'adj_matrix': adj_matrix, 'traffic_matrix': traffic_matrix, 'nodes_params': nodes_params}

  return Graphs

path_to_folder = '/content/drive/MyDrive/Кола' # Свой путь
Graphs = get_graphs(path_to_folder)

In [6]:
Graphs['cola_test60'] # Выводим пример

{'adj_matrix': array([[ 0.,  0., 60., 60., 60.],
        [ 0.,  0., 60., 60., 60.],
        [60., 60.,  0., 60.,  0.],
        [60., 60., 60.,  0., 60.],
        [60., 60.,  0., 60.,  0.]]),
 'traffic_matrix': array([[ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [20.,  0.,  0.,  0.,  0.]]),
 'nodes_params': {'degrees': [np.float64(3.0),
   np.float64(3.0),
   np.float64(3.0),
   np.float64(4.0),
   np.float64(3.0)],
  'source_demands': [np.float64(20.0),
   np.float64(0.0),
   np.float64(20.0),
   np.float64(0.0),
   np.float64(20.0)],
  'target_demands': [np.float64(20.0),
   np.float64(40.0),
   np.float64(0.0),
   np.float64(0.0),
   np.float64(0.0)],
  'sum_demands': [np.float64(20.0),
   np.float64(20.0),
   np.float64(10.0),
   np.float64(0.0),
   np.float64(10.0)]}}

In [23]:
import networkx as nx
from networkx import *

def graph_topology(main_graph): # Для графа исходного считаем кол-во компонент связности, а после все для наибольшей компоненты
  topology = []

  number_of_connected_components = nx.number_connected_components(main_graph)

  if nx.is_connected(main_graph):
    graph = main_graph
  else:
    components = list(nx.connected_components(main_graph))
    largest_component = max(components, key=len)
    graph = main_graph.subgraph(largest_component)

  num_nodes = graph.number_of_nodes()
  topology.append(num_nodes)

  num_edges = graph.number_of_edges()
  topology.append(num_edges)

  topology.append(number_of_connected_components)

  degrees = graph.degree()
  avg_degree = np.mean(graph.degree(), axis = 0)[1]
  topology.append(avg_degree)

  diameter = nx.diameter(graph)
  topology.append(diameter)

  avg_shortest_path_length = nx.average_shortest_path_length(graph)
  topology.append(avg_shortest_path_length)

  avg_clustering_coefficient = nx.average_clustering(graph)
  topology.append(avg_clustering_coefficient)

  local_clustering = list(nx.clustering(graph).values())
  std_clustering_coefficient = np.std(local_clustering)
  topology.append(std_clustering_coefficient)

  node_betweenness = list(nx.betweenness_centrality(graph).values())
  min_node_betweenness = min(node_betweenness)
  topology.append(min_node_betweenness)
  max_node_betweenness = max(node_betweenness)
  topology.append(max_node_betweenness)
  avg_node_betweenness = sum(node_betweenness) / num_nodes
  topology.append(avg_node_betweenness)

  edge_betweenness = list(nx.edge_betweenness_centrality(graph).values())
  min_edge_betweenness = min(edge_betweenness)
  topology.append(min_edge_betweenness)
  max_edge_betweenness = max(edge_betweenness)
  topology.append(max_edge_betweenness)
  avg_edge_betweenness = sum(edge_betweenness) / num_edges
  topology.append(avg_edge_betweenness)

  L_abs = nx.laplacian_matrix(graph, weight=None)
  L_abs_dense = L_abs.toarray()
  abs_eigenvalues = np.sort(np.linalg.eigvalsh(L_abs_dense))
  absolute_spectral_radius = np.max(abs_eigenvalues)
  topology.append(absolute_spectral_radius)
  absolute_algebraic_connectivity = abs_eigenvalues[1]
  topology.append(absolute_algebraic_connectivity)

  L_norm = nx.normalized_laplacian_matrix(graph, weight=None)
  L_norm_dense = L_norm.toarray()
  norm_eigenvalues = np.sort(np.linalg.eigvalsh(L_norm_dense))
  normalized_spectral_radius = np.max(norm_eigenvalues)
  topology.append(normalized_spectral_radius)
  normalized_algebraic_connectivity = norm_eigenvalues[1]
  topology.append(normalized_algebraic_connectivity)

  return topology

def statistic_dataframe(Graphs): # Делаем сводную таблицу топологий графов
  Topologies = []
  for graph_name, graph_info in Graphs.items():
    adj_matrix = graph_info['adj_matrix']
    graph = nx.from_numpy_array(adj_matrix)
    Topologies.append(graph_topology(graph))

  Topology_labels = ['Number of Nodes', 'Number of Edges', 'Number of Connected Components',
            'Average Node Degree', 'Diameter(hops)', 'Average Shortest Path Length (Hops)', 'Average Clustering Coefficient', 'Standard Deviation of Clustering Coefficient',
            'Minimum Node Betweenness Centrality', 'Maximum Node Betweenness Centrality', 'Average Node Betweenness Centrality',
            'Minimum Edge Betweenness Centrality', 'Maximum Edge Betweenness Centrality', 'Average Edge Betweenness Centrality',
            'Absolute Spectral Radius', 'Absolute Algebraic Connectivity', 'Normalized Spectral Radius', 'Normalized Algebraic Connectivity']

  Table = pd.DataFrame(Topologies, index = Graphs.keys(), columns = Topology_labels)
  Table = Table.sort_index(key=lambda idx: pd.MultiIndex.from_tuples([(s[:4], s[5:]) for s in idx]))

  return Table

Topology_table = statistic_dataframe(Graphs) # Тут используем уже словарь графов

In [24]:
Topology_table # Смотрим сводную таблицу

,Number of Nodes,Number of Edges,Number of Connected Components,Average Node Degree,Diameter(hops),Average Shortest Path Length (Hops),Average Clustering Coefficient,Standard Deviation of Clustering Coefficient,Minimum Node Betweenness Centrality,Maximum Node Betweenness Centrality,Average Node Betweenness Centrality,Minimum Edge Betweenness Centrality,Maximum Edge Betweenness Centrality,Average Edge Betweenness Centrality,Absolute Spectral Radius,Absolute Algebraic Connectivity,Normalized Spectral Radius,Normalized Algebraic Connectivity
2024_north,73,129,1,3.534247,12,4.932648,0.237900,0.288613,0.000000,0.249449,0.055389,0.001078,0.158177,0.038238,7.969051,0.083638,1.886968,0.023471
2024_south,73,126,1,3.452055,11,5.126332,0.224658,0.264835,0.000000,0.281282,0.058117,0.000571,0.223187,0.040685,8.042321,0.073782,1.863223,0.020299
2024_southwest,50,82,1,3.280000,10,4.435918,0.149143,0.204488,0.000425,0.296565,0.071582,0.001905,0.239233,0.054097,8.273007,0.096713,1.877175,0.028854
cola_large,1287,1655,7,2.571873,43,16.686631,0.053009,0.176974,0.000000,0.373196,0.012207,0.000001,0.274101,0.010083,16.174495,0.002005,1.992690,0.000783
cola_t1,667,820,2,2.458771,41,15.657122,0.012627,0.087698,0.000000,0.372381,0.022041,0.000020,0.122714,0.019094,14.268063,0.006159,1.990700,0.002408
cola_t10,83,98,1,2.361446,16,7.533059,0.000000,0.000000,0.001581,0.364927,0.080655,0.008130,0.262785,0.076868,6.630519,0.034313,1.986131,0.014713
cola_t2,32,47,3,2.937500,7,3.266129,0.075000,0.189846,0.000000,0.269211,0.075538,0.017641,0.142910,0.069492,8.290954,0.254758,1.876112,0.094280
cola_t3,21,33,1,3.142857,6,2.738095,0.092063,0.120363,0.003947,0.268697,0.091479,0.032625,0.191190,0.082973,6.927328,0.340390,1.816175,0.121752
cola_t5,405,559,4,2.760494,41,15.477729,0.080329,0.201639,0.000000,0.385620,0.035925,0.000012,0.346548,0.027688,7.504335,0.003941,1.978589,0.001426
cola_t6,405,559,4,2.760494,41,15.477729,0.080329,0.201639,0.000000,0.385620,0.035925,0.000012,0.346548,0.027688,7.504335,0.003941,1.978589,0.001426
